# 

title: Water Rights Restored to the Gila River

subtitle: The impacts of irrigation on vegetation health in the Gila
River Valley

author:

-   Elsa Culler

-   Nate Quarderer

date: last-modified

image: /img/earth-analytics/water-rights/lesson-water-rights.png

image-alt: “Dry river with dead plants turns into a stream with living
plants”

description: \|

In 2004, the Akimel O’‘otham and Tohono O’’odham tribes won a water
rights settlement in the US Supreme Court. Using satellite imagery, we
can see the effects of irrigation water on the local vegetation.

learning-goals:

-   Open raster or image data using code

-   Combine raster data and vector data to crop images to an area of
    interest

-   Summarize raster values with stastics

-   Analyze a time-series of raster images

params:

id: stars

site_name: Gila River Indian Community

event: water rights case

data_dir: gila-river

jupyter:

kernelspec:

    name: learning-portal

    language: python

    display_name: Learning Portal

# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

Note: For some reason I cannot get ndvi_diff to properly store. If using ndvi_diff in this notebook just recalculate it. 

In [1]:
%store -r

Unable to restore variable 'ndvi_diff', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>
Unable to restore variable 'ndvi_diff_plot', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>


You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [3]:
####Just import everything I already listed in the 00-setup notebook#### 

import json
from glob import glob

# do use pathlib as we are downloading and calling data for this assignment

from pathlib import Path
import os

# import earthpy and pandas as pd & geopandas as gpd. Also geopandas is useful for working with the zipfiles.
import earthpy as py
import pandas as pd 
import geopandas as gpd
#To help with plotting we will need the following packages for raster data. Hvplot.xarray allows for interactive ploting. Matplotlib packages can help make our final figures look much nicer, so we can include those as well. 

#Packages for plotting and working with raster data (remember we are using pandas and not earthpy):
import hvplot.xarray
import hvplot.pandas
import rioxarray as rxr
import xarray as xr

#Packages for editing the style of our later figures:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap # color ramps
from matplotlib.patches import Patch # legend for final figure

# For the sample data use the two links from figshare (https://ndownloader.figshare.com/files/54896600 AND https://ndownloader.figshare.com/files/55242452) 
# For the data downloads skip to the two notebooks on vegetation-91 and vegetation-92
# Set these up in my local directory (desktop folder earth-analytics)
# Since we are pulling data from zipfiles I will need another library for that
import zipfile 
import numpy as np

CONFUSION NOTE: Due to some syntax issues in the notebooks I am having trouble understanding which events we are tracking? In the notebooks we looked at change from 2018_2020 and then 2021_2023. In the below instructions, step 4 reads with ?meta:params.site_name unspecified. On the earth data science learning portal it was using the water rights case, but we did not follow those dates so I am just a little lost...

# STEP 4: Is the NDVI different within the **?meta:params.site_name** after the **?meta:params.event**?

You will compute the mean NDVI inside and outside the fire boundary.
First, use the code below to get a `GeoDataFrame` of the area outside
the Reservation.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Check the variable names - Make sure that the code uses your
boundary <code>GeoDataFrame</code></li>
<li>How could you test if the geometry was modified correctly? Add some
code to take a look at the results.</li>
</ol></div></div>

In [ ]:
# I will create the new ndvi calculations to do this section of the notebook
# I will need to use some of my odd vars for this.
    # This includes:
        # ndvi_scaled → 3D DataArray with dims (date, y, x) and values in [–1,1] for ALL dates/years (154)
        # boundary_gdf → dissolved Gila River boundary (AIANNHCE == '1310'), same CRS as NDVI

In [6]:
# Compute the area outside the fire boundary
# So this means treating one area as district 1-7, like with my Gila_subdivisions var
# The other areas would be anything other than those districts

# Calculating the area from 2001 to 2023 in Gila

# Project boundary to an equal-area projection for accurate area
boundary_eq = boundary_gdf.to_crs(epsg=5070)

# Calculate area in square kilometers
boundary_eq["area_km2"] = boundary_eq.geometry.area / 1e6

boundary_eq[["area_km2"]]


,area_km2
0,1511.981966


In [ ]:
# Then we can float and call the area var for inside Gila 
# I can also give this a label for the print so it is easy to know what the area refers to

gila_area_km2 = float(boundary_eq["area_km2"].iloc[0])
print("Gila River Indian Community area (km²):", gila_area_km2)

Gila River Indian Community area (km²): 1511.9819659021343


Next, clip your DataArray to the boundaries for both inside and outside
the reservation. You will need to replace the `GeoDataFrame` name with
your own. Check out the [lesson on clipping data with the `rioxarray`
library in the
textbook](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/).

> **GOTCHA ALERT**
>
> It’s important to use `from_disk=True` when clipping large arrays like
> this. It allows the computer to use less valuable memory resources
> when clipping - you will probably find that otherwise the cell below
> crashes your kernel.

In [ ]:
# Before this next step I need to check that the ndvi_scaled knows its CRS

ndvi_scaled.rio.crs

# In the long output it provides the CRS is WGS 84 (EPSG:4326).
# This is important as the boundary var and ndvi_scaled need the same CRS to clip correctly

CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]')

In [ ]:
# Clip data to both inside and outside the boundary

# Clip data from 2001 to 2023 in Gila 
# NDVI inside Gila (same shape, pixels outside = NaN)
# We will "mask" everything outside this boundary using drop=False
# When I ran this code the first time (2 code cells down) the boundary_gdf.geometry was being interpreted as a featurecollection which rasterio does NOT like
# In the second run of this code (next cell) we can fix this issue by giving rasterio a single geometry 
# To do this we need a new library from shapely.geometry import mapping 

# Import needed library for single geometry 
from shapely.geometry import mapping 

In [12]:
# Create the single geometry

# Merge all Gila polygons into one geometry
# unary_union is deprecating so we need to use a different function for this
# Python said to do the following: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.

gila_geom = boundary_gdf.union_all()

#### What union_all() function does ####
# Takes all geometries in my gdf and combines them into a single geometry

# Convert to GeoJSON-like mapping and wrap in a list for rioxarray so it actually likes this
gila_geoms = [mapping(gila_geom)]

In [13]:
#### CONT ####
# NOW we can clip data to both inside and outside the boundary

# Same notes still apply: 
    # NDVI inside Gila (same shape, pixels outside = NaN)
    # We will "mask" everything outside this boundary using drop=False

ndvi_inside = ndvi_scaled.rio.clip(
    gila_geoms,
    boundary_gdf.crs,
    drop=False
)

# NDVI outside Gila (invert the above mask) using .isnull
ndvi_outside = ndvi_scaled.where(ndvi_inside.isnull())

In [ ]:
# So the difference between these two vars is: 
    #ndvi_inside -> NDVI over time inside Gila only
    #ndvi_outside -> NDVI over time outside Gila only
# So of these vars still include out dimensions of date, x, y for the data retained in them (i.e., whatever we DIDNT mask)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>For <strong>both inside and outside</strong> the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary:</p>
<ul>
<li>Group the data by year</li>
<li>Take the mean. You always need to tell reducing methods in
<code>xarray</code> what dimensions you want to reduce. When you want to
summarize data across <strong>all</strong> dimensions, you can use the
<code>...</code> syntax, e.g. <code>.mean(...)</code> as a
shorthand.</li>
<li>Select the NDVI variable</li>
<li>Convert to a DataFrame using the <code>to_dataframe()</code>
method</li>
<li>Join the two DataFrames for plotting using the <code>.join()</code>
method. You will need to rename the columns using the
<code>lsuffix=</code> and <code>rsuffix=</code> parameters</li>
</ul>
<p>Finally, plot annual July means for both inside and outside the
Reservation on the same plot.</p></div></div>

> **GOTCHA ALERT**
>
> The DateIndex in pandas is a little different from the Datetime
> Dimension in xarray. You will need to use the `.dt.year` syntax to
> access information about the year, not just `.year`.

In [16]:
# Compute mean annual July NDVI
# So we need:
    # Only July dates
    # Then spatial mean per date
    # Then annual mean of July NDVI per year

# Need to make sure our date values are set for both inside and outside Gila 

ndvi_inside = ndvi_inside.assign_coords(
    date=pd.to_datetime(ndvi_inside["date"].values)
)
ndvi_outside = ndvi_outside.assign_coords(
    date=pd.to_datetime(ndvi_outside["date"].values)
)

# Now we can select for July specifically 
# To do this we will be masking again. Essentially anything that isn't July gets masked from the array 
# So we still use our "date" value but make it only equal to 7 (or July)

# Boolean mask for July, set July and then mask
july_mask_inside = ndvi_inside["date"].dt.month == 7
july_mask_outside = ndvi_outside["date"].dt.month == 7

ndvi_inside_july = ndvi_inside.isel(date=july_mask_inside)
ndvi_outside_july = ndvi_outside.isel(date=july_mask_outside)

# Now its time to calculate the spatial means for these 
# Mean NDVI per July date, inside vs outside
inside_july_ts = ndvi_inside_july.mean(dim=("y", "x"), skipna=True)
outside_july_ts = ndvi_outside_july.mean(dim=("y", "x"), skipna=True)

# Now that we have all our July means:
# Group by year and take the mean of all July dates in that year
inside_july_annual = inside_july_ts.groupby("date.year").mean(dim="date")
outside_july_annual = outside_july_ts.groupby("date.year").mean(dim="date")

inside_july_annual
outside_july_annual

<xarray.DataArray 'NDVI' (year: 22)> Size: 88B
array([0.25820887, 0.22367078, 0.22996643, 0.22637603, 0.255579  ,
       0.22809869, 0.21077679, 0.23958243, 0.21148   , 0.23956761,
       0.22009781, 0.219941  , 0.2224586 , 0.21858576, 0.21890882,
       0.2066194 , 0.22712648, 0.20656556, 0.22747007, 0.21932867,
       0.2266497 , 0.20914489], dtype=float32)
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B 0
  * year         (year) int64 176B 2001 2002 2003 2004 ... 2019 2020 2021 2022

In [ ]:
# You will notice that our array instead of having 'NDVI' date 154 x: some number, y: some number, is now just (year:22). 
# This is good because it means the array now has yearly july mean NDVI data for 22 years (2001-2022).

Now, take the difference between outside and inside the site boundary
and plot that. What do you observe? Don’t forget to write a headline and
description of your plot!

In [17]:
# Plot difference inside and outside the boundary
# We will essentially be making a time series here
# We need to create a difference variable first 
july_ndvi_diff = inside_july_annual - outside_july_annual
july_ndvi_diff.attrs["long_name"] = "Mean July NDVI difference (inside - outside)"
july_ndvi_diff.attrs["units"] = "unitless NDVI"

In [35]:
# Holoviews for exporting plots
import holoviews as hv

In [42]:
#### CONT ####
# Plot difference inside and outside the boundary
# We can create two different plots 
    # 1 -> plot inside, outside, and difference (hvplot)
    # 2 -> plot just the difference (inside – outside)

#We can run the first one here 
# Inside vs outside July NDVI over time

# Lets add a good title so we know we are looking at the NDVI scores for each area
# For our two plotted lines in this graph we can also adjust the labels (which I am truncating to Gila so the figure isn't so stretched), line color and line weight
inside_outside_plot = (
    inside_july_annual.hvplot.line(
        label="Inside Gila",
        xlabel="Year",
        ylabel="Mean July NDVI",
        title="Mean July NDVI Inside vs Outside Gila (2001–2022)",
        line_width=3,
        color="green",
        grid=True,
        legend="top_left"
    )
    *
    outside_july_annual.hvplot.line(
        label="Outside Gila",
        line_width=3,
        color="blue"
    )
)

inside_outside_plot

hv.save(inside_outside_plot, "july_ndvi_inside_outside.html")

In [43]:
# Now we can call the inside_outside_plot
inside_outside_plot

:Overlay
   .Curve.Inside_Gila  :Curve   [year]   (NDVI)
   .Curve.Outside_Gila :Curve   [year]   (NDVI)

### Understanding the first plot (Mean July NDVI Inside vs. Outside Gila (2001-2022))

The first thing to notice is that for all the NDVI mean values, regardless of if they are inside or outside Gila, are positive in July. This indicates that peak-season vegetation greenness is reliably present across the region. However, the magnitude of July NDVI differs substantially between the two areas.

Second, across the entire 2001–2022 period, the outside Gila region displays higher mean July NDVI than the inside Gila region. The outside area generally ranges from approximately 0.21 to 0.26, showing moderate but stable July greenness. In contrast, NDVI values inside Gila remain consistently below even the lowest values observed outside the boundary (with the only exception being 2005 where inside Gila has a mean July NVDI of almost 0.24).

More importantly, these lines NEVER intersect. Inside Gila never exceeds outside Gila in July during this period. Even after the 2004 water rights settlement, there is no clear shift in mean summer NDVI within the Gila boundary. This suggests that, at least for July greenness, the vegetation response did not show a dramatic or immediate increase even nearly two decades after this policy change.

Of course, there are a lot of mediating variables to consider. A big one is the affect of climate change and the general availability of water in the region. It is also possible that due to the long deprevation of water rights, that topographically, inside Gila and outside Gila have distinct differences due to human incursions. This could include mediating factors like soil type differences, localized irrigation, cropping patterns, drought exposure, or land use constraints. Any one of these can limit vegetation greenness inside the boundary compared to surrounding lands, even with improved water access. A fuller interpretation would require connecting NDVI trends to hydrologic data or land management practices but the pattern we see in this first plot is clear: July NDVI remains consistently higher outside the Gila boundary throughout 2001–2022.

But we should also be CAUTIOUS in this sort of interpretation. Just because the returning of water rights to the Gila River Indian Community did not create a positive vegetation effect DOES NOT MEAN we should be against returning land and water rights to the many sovereign nations across the US. It is very easy to weaponize this kind of data against tribal sovereignty. Often researchers and scientists view their data or their science as "valueless"-- its just numbers, observations, and so on. As a teacher of research methods, I always highlight that data always inherently has value (and often political value) that we must always be aware of. We see the consequences of this in how data can be interpreted, misinterprered and disinterpreted (if we borrow from the mis- v. disinformation narrative). So while on the surface there does not seem to be an ENVIRONMENTAL benefit to the return of water rights (from this very simple, one dimensional plot), there are surely SOCIAL and CULTURAL benefits that are unseen. 

In [44]:
# Plotting just the difference
july_diff_plot = july_ndvi_diff.hvplot.line(
    xlabel="Year",
    ylabel="Inside – Outside NDVI",
    title="Mean July NDVI Difference (Gila vs. Outside, 2001–2022)",
    line_width=3,
    color="purple",
    grid=True
)

july_diff_plot

hv.save(july_diff_plot, "july_ndvi_inside_outside_difference.html")


In [45]:
# Call the second plot
july_diff_plot

:Curve   [year]   (Mean July NDVI difference (inside - outside))

### Understanding the second plot (Mean July NDVI difference (Gila v. outside, 2001-2022))

For anyone like me who is mathematically challenged taking the difference between inside and outside the Gila boundary can result in some headscratching. 

First, we need to consider what each of the values in this graph represent and why. For each year, every NDVI value is the (Mean July NDVI inside Gila) minus (Mean July NDVI outside Gila). Essentially what this graph lets us answer is within each given year from 2001 to 2022, how much greener was the inside of the Gila boundary to its surroundings? The short answer is that inside Gila was never greener than outside (just like we see in plot 1).

The long answer: You will notice immediately that in comparison to the first plot, this line starts BELOW 0 and never crosses 0, and it is always a negative NDVI value. However, unlike the first plot, this negative NDVI value goes from about -0.05 in 2001 to -0.02 in 2022. So we are getting closer to 0, which means the greenness within the Gila boundary is becoming SIMILARLY green to its surroundings. So while plot 1 showed us that the vegegatation inside Gila is consistently less greener (so less productive, less biomass, less photosynthesis) than outside, this gap is NARROWING when looking at plot 2. 

This narrowing gap may suggest a few things, and also should cause us to think about other contributing factors (and counterfactuals):

1) That vegetation conditions inside Gila are *in fact* improving
--> For example, even though absolute NDVI remains lower than outside, the relative trend indicates reduced vegetation stress or increased productivity inside the boundary.

2) There may be shifts in land use or management contributing to this improvement (OR) the returning of water rights made their burden on the environment *less*
--> For example, factors such as irrigation expansion, riparian restoration, crop type changes, or post-drought recovery could narrow the NDVI gap.

3) There are other changing external conditions related to complex factors, like climate change, that may actually be affecting the *areas outside Gila more NEGATIVELY* 
--> For example, the gap may be narrowing because the areas surrounding Gila are doing worse because of increasing drought stress, loss of water rights, land degradation, or changing agricultural practices. In any case, in this scenario the surrounding landscape becomes less green in July, reducing the inside–outside difference.

4) Most complex reason could be that between the two areas that there is increasing environmental similarity
--> For example, broader climatic or hydrological changes may be influencing both areas in ways that make them converge, and this has nothing to do with water rights. 

In all of these possible scenarios or reasons for the narrowing gap, it is obvious that we do not have enough data to conclude whether this narrowing gap is due to water rights or not. However, the plot does clearly show a long-term narrowing of the vegetation greenness gap between the inside and outside of the Gila boundary in July.


In [40]:
# For putting this on the portfolio I want to combine these figures to save me formatting headaches. 
# I use cols(1) so they are stacked top to bottom rather than side by side. 

combined_layout = (inside_outside_plot + july_diff_plot).cols(1)

combined_layout

hv.save(combined_layout, "july_ndvi_inside_outside_side_by_side.html")

In [41]:
# Call the combined version
combined_layout

:Layout
   .Overlay.I :Overlay
      .Curve.Inside_Gila  :Curve   [year]   (NDVI)
      .Curve.Outside_Gila :Curve   [year]   (NDVI)
   .Curve.I   :Curve   [year]   (Mean July NDVI difference (inside - outside))

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [46]:
%store gila_area_km2 gila_geom gila_geoms inside_july_annual outside_july_annual  ndvi_inside ndvi_outside ndvi_inside_july ndvi_outside_july

Stored 'gila_area_km2' (float)
Stored 'gila_geom' (Polygon)
Stored 'gila_geoms' (list)
Stored 'inside_july_annual' (DataArray)
Stored 'outside_july_annual' (DataArray)
Stored 'ndvi_inside' (DataArray)
Stored 'ndvi_outside' (DataArray)
Stored 'ndvi_inside_july' (DataArray)
Stored 'ndvi_outside_july' (DataArray)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!